In [1]:
import numpy as np
import ccdredux as ccd
import imfuncs as imf
from astropy.io import fits as pf
from scipy.ndimage import filters
import shutil
from math import fabs
import glob

In [2]:
""" 
Define a function to create a temporary sky frame from
three input frames
"""

def makesky_3(infiles, medians, indices):
    """ Makes a temporary sky file from 3 input files """
    
    tmpdat = pf.getdata(infiles[indices[0]])
    tmpshape = tmpdat.shape
    tmpsky = np.zeros((3,tmpshape[0],tmpshape[1]))
    for i in range(3):
        tmpsky[i,:,:] = pf.getdata(infiles[indices[i]]) / medians[indices[i]]
    outsky = np.median(tmpsky,axis=0)
    del tmpdat,tmpsky
    return outsky

In [55]:
""" 
A function that makes copies of the raw data files that we can freely modify.
Unfortunately, something is screwed up with the header in the raw files,
so we are just going to copy over the data, which is non-optimal
"""
def get_raw(sciframes):
    """ 
    Inputs:
      sciframes - list or array containing the frame numbers associated with
                  the observations of the desired object
      lensroot  - rootname for output files
    """
    rawdir = '../Raw_scam/'
    rawroot = 'jun18i0'   
    infiles = []
    for i in sciframes:
        rawname = '%s%s%03d.fits' % (rawdir,rawroot,i)
        workname = 'work_%03d.fits' % i
        infiles.append(workname)
        #shutil.copyfile(rawname,workname)
        data = pf.getdata(rawname,ignore_missing_end=True)
        pf.PrimaryHDU(data).writeto(workname)
        del data
    return infiles

In [88]:
""" Get the raw data (modify for each lens-filter combination) """
sciframes = np.arange(229,238)
lensroot = 'F2214_Kp'
infiles = get_raw(sciframes)
print ''
print infiles

  4�  /  +  7�  8�  -�  7 [astropy.io.fits.card]
  5  7,  64  8�  7  9�  :P  8;  ;f  7�  5  7�  5V  7L  4I  7W  9�  :$  6D  34 [astropy.io.fits.card]
  4l  ,d  6'  :c  8P  <�  67  9�  8V  9�  7a  4�  9  4�  4  7�  7�  6>  7�  6< [astropy.io.fits.card]
  5W  6{  4�  7h  7J  5&  9   7�  7�  6�  8�  ;�  6�  6a  2�  4K  7�  4R  6�  5 [astropy.io.fits.card]
  1F  4�  6e  :�  <.  6�  9�  5�  5F  ;k  6�  :�  :.  3�  ;C  3x  7�  7�  9�  9X [astropy.io.fits.card]
  7�  ::  7�  8\  7   7�  8_  5  4R  9�  ;�  1z  3]  >%  8�  4F  8}  9�  ;�  :� [astropy.io.fits.card]
  82  3*  7�  8  6  7  <�  >�  8�  3  8�  5�  8�  7   9�  ;�  9I  6�  >-  7k [astropy.io.fits.card]
  7�  7�  5�  9  9�  8�  8  :  6�  9�  ;d  5�  9  5�  8�  9P  :�  7R  <~  ;_ [astropy.io.fits.card]
  3|  <�  :  :�  7;  :  7V  7�  3@  ;"  4�  7{  <  :�  8�  ;�  :�  8�  4W  7O [astropy.io.fits.card]
  5�  8f  8  5:  <  ;}  <)  =)  8b  :�  8�  8�  <�  5�  3b  9=  6�  :  6�  :+ [astropy.io.fits.card]
  7i  7  6�


['work_229.fits', 'work_230.fits', 'work_231.fits', 'work_232.fits', 'work_233.fits', 'work_234.fits', 'work_235.fits', 'work_236.fits', 'work_237.fits']


  24  6  8Z  9�  7�  6H  3�  /�  3�  2�  6$  1�  5N  2�  -E  )  5�  7@  ,Z  4� [astropy.io.fits.card]
  3|  5  4/  6s  5  7�  8  65  9  5P  3  5~  2�  4�  1�  5�  7h  8$  4&  1 [astropy.io.fits.card]
  2b  *�  46  8r  5�  :}  4  7Z  6	  7�  4�  2�  6�  2�  1�  5�  6  3�  5v  3� [astropy.io.fits.card]
  3  4�  2�  5m  5  3  6�  5�  5�  4s  6)  8�  4p  4X  0r  2L  5�  2   4�  2� [astropy.io.fits.card]
  0  3  4�  8�  9�  4�  6�  3�  3)  9�  3�  8�  7�  2F  9  1(  5#  5x  7�  6� [astropy.io.fits.card]
  5�  8Z  5Q  6l  4�  5  6f  2�  2|  7�  9d  /s  15  ;<  6v  2)  6+  7�  9�  8� [astropy.io.fits.card]
  5�  0�  5L  6<  3�  4�  :M  <�  5�  0�  6�  3n  6�  5  75  9�  6�  4e  ;�  4� [astropy.io.fits.card]
  5  5�  3�  6�  7^  7#  5�  8  4�  7�  8�  4  6�  3�  6:  6�  8r  5W  :�  8� [astropy.io.fits.card]
  16  :  8B  8�  5c  7�  5  5�  1�  8�  23  5�  9�  8l  6d  9|  8[  6Z  2  5N [astropy.io.fits.card]
  3�  6w  5�  2�  :  9   9^  :�  6:  8�  6O  6�  :5  3  1F  6�

In [89]:
""" Make the first sky frame """
skyname = '%s_sky1.fits' % lensroot
ccd.median_combine(infiles,skyname,normalize=True)

median_combine: Inputs:
-----------------------
  bias frame: [No bias file]

median_combine: Loading files
-----------------------------
 work_229.fits
 work_230.fits
 work_231.fits
 work_232.fits
 work_233.fits
 work_234.fits
 work_235.fits
 work_236.fits
 work_237.fits

median_combine: Getting info on first file
------------------------------------------
Filename: work_229.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   int32   

median_combine: setting up stack for images (HDU 0)
----------------------------------------------------
Stack will have dimensions (9,256,256)
 work_229.fits
    Normalizing work_229.fits by 14659.000000
 work_230.fits
    Normalizing work_230.fits by 16401.000000
 work_231.fits
    Normalizing work_231.fits by 17463.000000
 work_232.fits
    Normalizing work_232.fits by 17645.000000
 work_233.fits
    Normalizing work_233.fits by 16770.500000
 work_234.fits
    Normalizing work_234.fits by 

In [90]:
""" 
Use the sky frame to make an initial bad pixel mask and then
update the sky frame itself to mask out the bad pixels.
"""
sky = imf.Image(skyname)

""" 
Do a 3-sigma clipping on the data and use the resulting clipped
mean and clipped rms to set the criterion for determining bad 
pixels
"""
sky.sigma_clip()
diff = np.fabs((sky.hdu[0].data - sky.mean_clip) / sky.rms_clip)

""" Create the bad pixel mask and write it out """
bpmask = diff>5.
tmp = bpmask.astype(int)
maskname = '%s_mask_sky1.fits' % lensroot
pf.PrimaryHDU(tmp).writeto(maskname,clobber=True)
del tmp

""" Replace the bad pixels with the median value in the image """
skydat = sky.hdu[0].data.copy()
skymed = np.median(skydat)
skydat[bpmask] = skymed

""" Save the result """
sky1v2name = '%s_sky1_v2.fits' % lensroot
pf.PrimaryHDU(skydat).writeto(sky1v2name,clobber=True)


Loading file F2214_Kp_sky1.fits
-----------------------------------------------
Filename: F2214_Kp_sky1.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU       6   (256, 256)   float64   
get_wcs: No valid WCS information in file header


In [91]:
""" 
Replace the bad pixels in the input files using a 2-step process:
  1. Replace each of the bad pixels with the overall median value
  2. WAIT FOR NOW ON THIS

"""
for i in infiles:
    hdu = pf.open(i,mode='update')
    data = hdu[0].data
    datmed = np.median(data)
    data[bpmask] = datmed
    hdu.flush()
    del(hdu)


In [92]:
""" Do a running sky subtraction """

""" Start by reading in all the files and calculating their median values """
alldat = np.zeros((len(infiles),bpmask.shape[0],bpmask.shape[1]))
allmed = np.zeros(len(infiles))
index_list = []
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    allmed[i] = np.median(tmp)
    if i==0:
        index_list.append([0,1,2])
    elif i==(len(infiles)-1):
        index_list.append([i-2,i-1,i])
    else:
        index_list.append([i-1,i,i+1])
for i in range(len(infiles)):
    tmp = pf.getdata(infiles[i])
    tmpsub = tmp - allmed[i] * makesky_3(infiles,allmed,index_list[i])
    outname = 'tmpsub_%02d.fits' % sciframes[i]
    pf.PrimaryHDU(tmpsub).writeto(outname,clobber=True)
        

In [93]:
""" Do the initial sky subtraction using the updated sky frame """
sky1 = pf.getdata(sky1v2name)
skymed = np.median(sky1)
for i in infiles:
    scidat = pf.getdata(i).astype(float)
    scimed = np.median(scidat)
    scidat[bpmask] = scimed
    scimed2 = np.median(scidat)
    diff = scidat - (scimed2 / skymed) * sky1
    outfile = i.replace('work','sub1')
    pf.PrimaryHDU(diff).writeto(outfile)
    print 'Wrote sky-subtracted image (pass 1) to %s' % outfile
    del scidat,diff

Wrote sky-subtracted image (pass 1) to sub1_229.fits
Wrote sky-subtracted image (pass 1) to sub1_230.fits
Wrote sky-subtracted image (pass 1) to sub1_231.fits
Wrote sky-subtracted image (pass 1) to sub1_232.fits
Wrote sky-subtracted image (pass 1) to sub1_233.fits
Wrote sky-subtracted image (pass 1) to sub1_234.fits
Wrote sky-subtracted image (pass 1) to sub1_235.fits
Wrote sky-subtracted image (pass 1) to sub1_236.fits
Wrote sky-subtracted image (pass 1) to sub1_237.fits


In [94]:
"""
Define a function that does a quick coadd of a list of input files given
pixel offsets between them.  The quick-and-dirty aspect to this processing
is that the function will just do integer pixel shifts.
"""

def quick_coadd(filelist, offsets, outfile):
    
    """ Start by shifting the offsets to be centered on the mean offset """
    dx = offsets[:,0] - (offsets[:,0].mean())
    dy = offsets[:,1] - (offsets[:,1].mean())
    dxrange = (dx.min(),dx.max())
    dyrange = (dy.min(),dy.max())
    
    """ Make a blank image of the appropriate size """
    dat0 = pf.getdata(filelist[0])
    xsize,ysize = dat0.shape[1],dat0.shape[0]
    del dat0
    outxsize = int(xsize + fabs(dxrange[0]) + fabs(dxrange[1]))
    outysize = int(ysize + fabs(dyrange[0]) + fabs(dyrange[1]))
    outim = np.zeros((outysize,outxsize))
    
    """ Insert the data with the appropriate offsets """
    x0 = fabs(dxrange[0])
    y0 = fabs(dyrange[0])
    x0 = dxrange[1]
    y0 = dyrange[1]
    x1 = (x0 - dx).astype(int)
    x2 = x1 + int(xsize)
    y1 = (y0 - dy).astype(int)
    y2 = y1 + int(ysize)
    print 'Output file will have size: %d x %d' % (outxsize,outysize)
    print x0,y0
    print dx,dy
    print x1,x2
    print y1,y2
    for i in range(len(filelist)):
        tmp = pf.getdata(filelist[i])
        print i, tmp.shape
        try:
            outim[y1[i]:y2[i],x1[i]:x2[i]] += tmp
        except:
            print 'Failed on image %i (%s) with x1=%d,x2=%d,y1=%d,y2=%d' % (i,filelist[i],x1[i],x2[i],y1[i],y2[i])
        del tmp
    pf.PrimaryHDU(outim).writeto(outfile,clobber=True)

In [96]:
""" Do a quick-and-dirty coadd """
offsets = np.loadtxt('%s_offsets.txt' % lensroot)
#offsets = np.loadtxt('J1618_Kp_offsets.txt')
xshifts = -1. * (offsets[:,0] - offsets[0,0])
yshifts = -1. * (offsets[:,1] - offsets[0,1])
subfiles = []
for i in infiles:
    subfiles.append(i.replace('work','sub1'))
print subfiles
outfile = '%s_coadd_quick.fits' % lensroot
print outfile
quick_coadd(subfiles,offsets,outfile)
#xshifts = offsets[:,0]
#yshifts = offsets[:,1]
#ccd.coadd_intshift(subfiles,xshifts,yshifts,outfile)
print ''
print 'Wrote coadded file to %s' % outfile

['sub1_229.fits', 'sub1_230.fits', 'sub1_231.fits', 'sub1_232.fits', 'sub1_233.fits', 'sub1_234.fits', 'sub1_235.fits', 'sub1_236.fits', 'sub1_237.fits']
F2214_Kp_coadd_quick.fits
Output file will have size: 279 x 280
11.7777777778 12.1111111111
[ -0.22222222 -11.22222222  11.77777778 -11.22222222  -0.22222222
  10.77777778 -11.22222222  11.77777778  -0.22222222] [  0.11111111  10.11111111   0.11111111 -10.88888889  12.11111111
 -10.88888889   0.11111111  11.11111111 -11.88888889]
[12 23  0 23 12  1 23  0 12] [268 279 256 279 268 257 279 256 268]
[12  2 12 23  0 23 12  1 24] [268 258 268 279 256 279 268 257 280]
0 (256, 256)
1 (256, 256)
2 (256, 256)
3 (256, 256)
4 (256, 256)
5 (256, 256)
6 (256, 256)
7 (256, 256)
8 (256, 256)

Wrote coadded file to F2214_Kp_coadd_quick.fits
